In [4]:
import pandas as pd
from sqlalchemy import select
from sqlalchemy.orm import sessionmaker

from shipping_cost_analysis.etl.db_tools import get_engine
from shipping_cost_analysis.models.schema import Customer, Transactions, Product, Region

In [6]:

engine = get_engine()
Session = sessionmaker(bind=engine)

with engine.connect() as conn:
    query = (
        select(
            Transactions.transaction_date,
            Transactions.customer_id,
            Transactions.stock_code,
            Transactions.invoice_no,
            Transactions.quantity,
            Transactions.unit_price,
            Transactions.sales,
            Customer.order_postal,
            Customer.order_city,
            Customer.order_state,
            Product.landed_cost,
            Product.shipping_cost_1000_r,
            Product.description,
            Product.category,
            Region.region
        )
        .join(Transactions, Transactions.stock_code == Product.stock_code)
        .join(Customer, Transactions.customer_id == Customer.customer_id)
        .join(Region, Region.order_state == Customer.order_state)
    )

    df = pd.read_sql(query, conn)

In [8]:
display(df.shape)
df.head()

(101054, 15)

,transaction_date,customer_id,stock_code,invoice_no,quantity,unit_price,sales,order_postal,order_city,order_state,landed_cost,shipping_cost_1000_r,description,category,region
0,2021-04-07 10:43:00,12347.0,22423,549222.0,1.0,76.5,76.5,94523,Pleasant Hill,CA,35.0,20.0,Taste of the Wild High Prairie Grain-Free Dry ...,Food,West
1,2021-04-07 10:43:00,12347.0,22423,549222.0,1.0,76.5,76.5,94523,Pleasant Hill,CA,35.0,20.0,Taste of the Wild High Prairie Grain-Free Dry ...,Food,West
2,2021-04-07 10:43:00,12347.0,22423,549222.0,1.0,76.5,76.5,94523,Pleasant Hill,CA,35.0,20.0,Taste of the Wild High Prairie Grain-Free Dry ...,Food,West
3,2021-04-07 10:43:00,12347.0,22423,549222.0,1.0,76.5,76.5,94523,Pleasant Hill,CA,35.0,20.0,Taste of the Wild High Prairie Grain-Free Dry ...,Food,West
4,2021-04-07 10:43:00,12347.0,22423,549222.0,1.0,76.5,76.5,94523,Pleasant Hill,CA,35.0,20.0,Taste of the Wild High Prairie Grain-Free Dry ...,Food,West


In [ ]:
price_variants = (
    df[(df["quantity"] == 1.0)]
    .groupby(["stock_code", "weight", "order_postal"])["unit_price"]
    .nunique()
    .reset_index(name="unique_prices")
)
price_variants[price_variants["unique_prices"] > 1].sort_values("stock_code")

In [ ]:
variants = (
    df[
        (df["quantity"] == 1.0)
        & (df["weight"] == 1.0)
        & (~df["unit_price"].isnull())
    ]
    .groupby(["stock_code"])["unit_price"]
    .unique()
    .reset_index(name="unit_price_variants")
)
variants[variants["unit_price_variants"].apply(len) > 1]

In [ ]:
"""

sns.boxplot(data=merged_orders, x="shipping_cost_1000_r")
plt.show()

g = sns.FacetGrid(
    merged_orders,
    col="region",
    col_wrap=4,
    height=4,
    sharex=True,
    sharey=True
)

g.map_dataframe(sns.scatterplot, x="shipping_cost_1000_r", y="unit_price", alpha=0.5)
g.set_titles(col_template="{col_name}")
g.set_axis_labels("Shipping Cost (1000 miles)", "Unit Price")
plt.subplots_adjust(top=0.9)
plt.show()

sns.scatterplot(data=merged_orders, x="landed_cost", y="unit_price")
sns.scatterplot(data=merged_orders, x="weight", y="unit_price")
sns.scatterplot(data=merged_orders, x="weight", y="shipping_cost_1000_r")








"""